### 依據原始的 predict/predict.py 修改而來

#### vvv Setting-1 vvvv

In [1]:
# prediction related paths, should be consistent with /preprocess/loadData.py and /train/trainModel.py
output_dim = 122
labelBinarizerPath = '../data/labelBinarizer_top{}.pickle'.format(output_dim)
savedModelH5 = '../train/trainModel_InceptionV3_1-v5_bird_frog_dog.h5'
spMapFile = '../data/species_mapping.csv'

In [2]:
from scipy import io
from scipy.io import wavfile
import numpy as np
import os
import pandas as pd
import pickle
import datetime
import time
from matplotlib import mlab
%matplotlib inline
np.random.seed(0)

In [3]:
# this parameter is used for preprocessing
# the number comes from the following equation: np.floor(sampling_frequency/(FFT_length-FFT_overlap))*num_of_seconds
# we use 16kHz sampling rate for the wavs, 512 FFT window length with 256 overlap and we investigate 5 seconds 
spectrogramWindowLength = int(5*np.floor(16000/(512-256)));
spectrogramHeight = 200

#### vvvvv 載入 labelBinarizer

In [4]:
lb = pickle.load(open(labelBinarizerPath, 'rb'))
classIdList = lb.inverse_transform(np.diag([1 for i in range(output_dim)]))
print(len(classIdList), '\n', classIdList)

122 
 ['ACCTRI' 'ACRCRI' 'ACTMOR' 'AEGCON' 'AETCHR' 'ALAGUL' 'ALCATT' 'ALCMOR'
 'ARBCRU' 'BABADE' 'BABOKI' 'BRAMON' 'BUEJAP' 'BUEOTA' 'BUEROB' 'BUFBAN'
 'CALCAL' 'CAPAFF' 'CINLEU' 'CUCOPT' 'CYARUF' 'DENCAN' 'DENFOR' 'DENLEU'
 'DICAEN' 'DICIGN' 'DICMAC' 'DICMIN' 'DUTMEL' 'EMBSPO' 'EOPMIG' 'EUDSCO'
 'FEJCAN' 'FEJKAW' 'FULFOR' 'GARCAN' 'GARTAE' 'GORMEL' 'HETAUR' 'HIRTAH'
 'HOPRUG' 'HORACA' 'HORBOR' 'HORFOR' 'HYLCHI' 'HYLGUE' 'HYLLAT' 'HYLTAI'
 'HYPAZU' 'HYPLEU' 'KALPUL' 'KUREIF' 'KURIDI' 'LANCRI' 'LIMFUJ' 'LIOSTE'
 'LITCAT' 'LOCALI' 'MEGERY' 'MICBUT' 'MICHEY' 'MICORN' 'MICSTE' 'ODOSWI'
 'ORITRA' 'OTUELE' 'OTULET' 'OTUSPI' 'PARHOL' 'PARMON' 'PASMON' 'PELFUK'
 'PHOAUR' 'PHYBIS' 'PHYINO' 'PITNYM' 'PNOFOR' 'POLBRA' 'POLMEG' 'POMMUS'
 'PRICRI' 'PRIFLA' 'PRIINO' 'PSINUC' 'PYCSIN' 'PYRERY' 'PYRNIP' 'RANLON'
 'RANSAU' 'RHAARV' 'RHAAUR' 'RHAMOL' 'RHAPRA' 'RHATAI' 'SCHBRU' 'SINWEB'
 'SITCAS' 'SITEUR' 'SPICHE' 'SPISEM' 'TARJOH' 'TRESIE' 'TURPAL' 'UROCAE'
 'UROSQU' 'YUHBRU' 'ZOSJAP' 'd00001' 'd00002'

#### vvvvv 載入原始物種對照表 species_mapping.csv

In [5]:
sp_map = pd.read_csv(spMapFile, sep='\t', encoding='utf-8')
sp_map

,category,classId,chineseName,scientificName,newCName,newCAliasName,newSPName,oldClassId
0,frog,HYLCHI,中國樹蟾,Hyla chinensis,中國樹蟾,NaN,Hyla chinensis,f00001
1,frog,KALPUL,亞洲錦蛙,Kaloula pulchra,亞洲錦蛙,花狹口蛙,Kaloula pulchra,f00002
2,frog,RHATAI,台北樹蛙,Rhacophorus taipeianus,臺北樹蛙,NaN,Rhacophorus taipeianus,f00003
3,frog,HYLTAI,台北赤蛙,Hylarana taipehensis,臺北赤蛙,NaN,Hylarana taipehensis,f00004
4,frog,MICSTE,史丹吉氏小雨蛙,Micryletta steinegeri,史丹吉氏小雨蛙,NaN,Micryletta steinegeri,f00005
5,frog,BUEOTA,太田樹蛙,Buergeria otai,太田樹蛙,NaN,Buergeria otai,f00006
6,frog,MICORN,小雨蛙,Microhyla ornata,小雨蛙,NaN,Microhyla ornata,f00007
7,frog,MICBUT,巴氏小雨蛙,Microhyla butleri,巴氏小雨蛙,NaN,Microhyla butleri,f00008
8,frog,HYLLAT,拉都希氏赤蛙,Hylarana latouchii,拉都希氏赤蛙,NaN,Hylarana latouchii,f00009
9,frog,POLMEG,斑腿樹蛙,Polypedates megacephalus,斑腿樹蛙,NaN,Polypedates megacephalus,f00010


#### vvvvv 載入 train 好的模型

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES']='0'
from keras.models import load_model
model = load_model(savedModelH5)

Using TensorFlow backend.


#### vvvvv 與 loadData 相同的音波處理程式碼

In [ ]:
# if 'x' array contains 1, this expands it inthe given directions
# used for the mask applied to the spectogram
def expandOnes(x, directions = [[-1,0], [1,0], [0,-1], [0,1]]):
    expand = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if (x[i,j] == 1):
                for direction in directions:
                    cx = i + direction[0]
                    cy = j + direction[1]
                    if (0 <= cx < x.shape[0] and 0 <= cy < x.shape[1]):
                        expand[cx,cy]=1;
    return x+expand;

In [ ]:
# function to filter the spectogram based on the energy of the signal
#
# parameters:
#   data: audio data
#   expandByOne: if it is True, than the mask of the spectogram will be expanded in every direction
#   dropZeroColumnsPercent: determines the ratio of 0 values along the frequency axis when a timeslice is dropped
#
# return values:
#   spectogram
#   filtered spectogram

def audioToFilteredSpectrogram(data, expandByOne = True, dropZeroColumnsPercent = 0.95):
    # calculate the spectogram
    
    #
    #tempSpec = np.log10(mlab.specgram(data, NFFT=512, noverlap=256, Fs=16000)[0])
    #vvvvv 在部份蛙音原始檔裡會出現導致 np.1og10 裡有 0 的情況產生, 而產生負無窮大的值, 故改成 +1 vvvvv
    tempSpec = np.log10(mlab.specgram(data, NFFT=512, noverlap=256, Fs=16000)[0]+1)

    # drop higher frequencies
    tempSpec = tempSpec[0:200,:]
    tempSpecFiltered = np.copy(tempSpec)

    # we analize the spectogram by 20x30 sized cells
    # to achieve better accuray the size of this cell should be finetuned
    rowBorders = np.ceil(np.linspace(0,tempSpec.shape[0], 20))
    columnBorders = np.hstack((np.ceil(np.arange(0,tempSpec.shape[1], 30)), tempSpec.shape[1]))
    rowBorders = [ int(x) for x in rowBorders ]
    columnBorders = [ int(x) for x in columnBorders ]
    keepCells = np.ones((len(rowBorders)-1, len(columnBorders)-1))

    # we create a mask for the spectogram: we scan the spectogram with the 20x30 sized
    # cell and create 0 mask based on the mean and std of the spectogram calculated for the cells and rows
    for i in range(len(rowBorders)-1):
        row_mean = np.mean(tempSpec[rowBorders[i]:rowBorders[i+1],:])
        row_std = np.std(tempSpec[rowBorders[i]:rowBorders[i+1],:])

        for j in range(len(columnBorders)-1):
            cell_mean = np.mean(tempSpec[rowBorders[i]:rowBorders[i+1],columnBorders[j]:columnBorders[j+1]])
            cell_max_top10_mean = np.mean(np.sort(tempSpec[rowBorders[i]:rowBorders[i+1],columnBorders[j]:columnBorders[j+1]], axis=None)[-10:])

            if (cell_mean < 0 or ((cell_max_top10_mean) < (row_mean + row_std)*1.5)):
                keepCells[i,j]=0

    # expand by ones (see above)
    if expandByOne:
        keepCells = expandOnes(keepCells)

    # apply the mask to the spectogram
    for i in range(keepCells.shape[0]):
        for j in range(keepCells.shape[1]):
            if not keepCells[i,j]:
                tempSpecFiltered[rowBorders[i]:rowBorders[i+1],columnBorders[j]:columnBorders[j+1]] = 0

    # drop zero columns
    # the amount of zero values along axis 0 (frequency) is calculated for every column (timeslice)
    # and it is dropped, if the number of zero values is higher than the dropZeroColumnsPercent 
    # eg. dropZeroColumnsPercent=0.95, than a column (timeslice) is dropped, if more than 95% of the values (frequencies) is 0
    tempSpecFilteredBackup = np.copy(tempSpecFiltered)
    tempSpecFiltered = np.delete(tempSpecFiltered, np.nonzero((tempSpecFiltered==0).sum(axis=0) > tempSpecFiltered.shape[0]*dropZeroColumnsPercent), axis=1)

    # if every row was 0 than use the backed up spectogram
    if tempSpecFiltered.shape[1] == 0:
        tempSpecFiltered = tempSpecFilteredBackup

    return tempSpec, tempSpecFiltered;

In [ ]:
# bulk processing of wav files in path
#
# parameters:
#   path        the source path
#   filenames   the filenames in the path
#   dontFilter  does not filter the spectogram if it is set to True
#
# return value:
#   specotogram data of multiple files

def wavsToSpectrogramByList(path, filenames, dontFilter=False):
#     print("wavsToSpectrogramByList...")
    data=list()
    for filename in filenames:
#         print('\r    Processing {}'.format(os.path.join(path, filename))),
        (tempSpecUnfiltered,tempSpecFiltered) = audioToFilteredSpectrogram(io.wavfile.read(os.path.join(path, filename))[1], expandByOne=True)
        if (not dontFilter):
            data.append(tempSpecFiltered)
        else:
            data.append(tempSpecUnfiltered)
#     print("\nwavsToSpectrogramByList finished")
    return data;

In [ ]:
# function to create training data from the list generated by wavsToSpectogramByList function
#
# parameters:
#   slist      the spectogram list generated by wavsToSpectogramByList function
#   N          (1*44100)/(1024-512)=86
#
# return values
#   X          the constructed input

  
def spectrogramListToT4_X(slist, N=spectrogramWindowLength): 
#     print("SpectrogramListToT4_X start...")

    rows = len(slist[0])
    X = np.empty((0,1,rows,N))

    # process all spectograms
    for i in range(len(slist)):
#         print('\r    Processing no. %d / %d' % (i, len(slist)))
        ranges = np.hstack((np.arange(0, len(slist[i][0]), N), len(slist[i][0])))

        for j in range(len(ranges)-1):
            # variable contains
            tempSpec = np.empty((1,rows,N))

            if (len(slist[i][0]) < N): # if data is shorter than N than fill up with zeros
                tempSpec[0] = np.hstack((slist[i],np.zeros((rows, N-len(slist[i][0])))))
            elif (ranges[j+1]-ranges[j] < N): # last element
                tempSpec[0] = slist[i][:,-N:]
            else: # other part of the spectrum
                tempSpec[0] = slist[i][:,ranges[j]:ranges[j+1]]

            X = np.vstack((X,[tempSpec]))
#     print("SpectrogramListToT4_X finished!")
    return X

#### vvvvv 載入音檔並轉換

In [ ]:
import os
import glob

# for data visiualiztion and GUI UX
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline

import librosa
import librosa.display
import tkinter as tk

In [ ]:
wave, fs = librosa.load('../live_demo/新月池語音_180424.wav', sr=None)

In [ ]:
frog_wave = ipd.Audio('../live_demo/新月池語音_180424.wav') # load a local WAV file
frog_wave

In [ ]:
import operator
def wavePredict(wavdirpath, fileName):
    tempSG = wavsToSpectrogramByList(wavdirpath, list([fileName]), dontFilter=False)
    X = spectrogramListToT4_X(tempSG, N=spectrogramWindowLength) #convert to t4
    spectroNo = len(X)
    print('spectroNo={}'.format(spectroNo))
    
    result = model.predict(X)
    result_argmax = np.argmax(result, axis=1)

    #將 numpy 值依個別出現次數由大到小排列
    unique, counts = np.unique(result_argmax, return_counts=True)
    sp_dict = dict(zip(unique, counts))
    sorted_sp_dict = sorted(sp_dict.items(), key=operator.itemgetter(1), reverse=True)
    
    print('這段音檔裡有可能的物種有:')
    retDict = dict()
    for i, sp in enumerate(sorted_sp_dict):
        sp_CName = sp_map[sp_map['classId']==classIdList[sp[0]]].chineseName.as_matrix()[0]
        probability = sp[1] / spectroNo
        retDict[sp_CName] = probability
        #print('  第 {} 可能為: {}'.format(i+1, sp_CName))
    print(retDict)

In [ ]:
FINAL_TEST_PATH = '../live_demo/'
for path, subdirs, files in os.walk(FINAL_TEST_PATH):
    i=1
    for name in files:
        if (name.endswith('.wav')):
            print('----------- test wav name#{}={}'.format(i,name))
            wavePredict(FINAL_TEST_PATH, name)
            i+=1

In [ ]:
'''Basic package'''
import os
# 告訴系統要第幾張卡被看到。 Ex. 硬體總共有8張顯卡，以下設定只讓系統看到第1張顯卡
# 若沒設定，則 Tensorflow 在運行時，預設會把所有卡都佔用
# 要看裝置內顯卡數量及目前狀態的話，請在終端機內輸入 "nvidia-smi"
# 若你的裝置只有一張顯卡可以使用，可以忽略此設定
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import queue
import cv2          #影像處理
import scipy.misc   #影像處理
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm #進度條
import matplotlib.pyplot as plt #繪圖
from sklearn.model_selection import train_test_split

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
from os import listdir
from os.path import isfile, isdir, join

SaveDirectory = os.getcwd()
print(SaveDirectory)

In [ ]:
label_dict = {'bird':0,
'bird_2':1,
'bird_picture':2,
'dog':3,
'frog':4,
'frog_picture':5,
}

for Class, label in label_dict.items():
    print(Class, label)
label_dict['bird_picture']

In [ ]:
# 指定要列出所有檔案的目錄
mypath = "/project/at071-group05/animals_sound/green_light/data_by_category/"
animal_item = 41
# 取得所有檔案與子目錄名稱
files = listdir(mypath)

Class_list = []
Class_label_list = []
Path_list = []
File_list = []

# 以迴圈處理
for f in files:
    # 產生檔案的絕對路徑
    fullpath = join(mypath, f)
    # 判斷 fullpath 是檔案還是目錄
    if isfile(fullpath):
        print("檔案：", f)
    elif isdir(fullpath):
        print("目錄：", f)
        sub_files = listdir(fullpath)
        for ff in sub_files:
            if ff != '.ipynb_checkpoints':
                Class_list.append(f)
                Class_label_list.append(label_dict[f])
                Path_list.append(fullpath + '/' + ff)
                File_list.append(ff)

In [ ]:
img_df = pd.DataFrame({'Class':Class_list, 'Class_label':Class_label_list ,'Path':Path_list, 'File':File_list})
img_df
#img_df = img_df.drop[0:8]
# img_df

'''
if img_df.["chineseName"].isin(['%d'], sp_CName)
  for i in range(len(img_df.index)):
       i = img_df.index
    return i
    
'''

In [ ]:
# PRINT IMAGE
print(img_df.Path.iloc[animal_item])
img = cv2.imread(img_df.Path.iloc[animal_item], 0)
plt.imshow(img, cmap='gray')
